In [1]:
results_path = "../benchmark-results/"

In [2]:
import glob

In [3]:
result_files = glob.glob(results_path + "*")

In [4]:
import pandas as pd
import json

In [5]:
results = []

for file in result_files:
    with open(file) as f:
        data = json.load(f)
    results.append(pd.json_normalize(data))

# Merging all results here

In [6]:
df = pd.concat(results, ignore_index=True)

In [7]:
df

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.max_cpu_utilization,run_stats.max_memory_utilization,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan
0,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-125-10,"DynamicList[task=CompSize,resource=Speed,cores...",True,104.405775,0.018969,0.011636,4870.725405,111558.897850,892.481093,...,0.852713,0.00000,0.361642,0.000000,16,0.412848,0.000000,0.631785,0.000000,NaN
1,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-125-100,"DynamicList[task=CompSize,resource=Speed,cores...",True,104.406135,0.018360,0.012827,4870.725405,111558.897850,892.570283,...,0.852713,0.00000,0.361641,0.000000,16,0.412847,0.000000,0.631783,0.000000,NaN
2,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-125-1000,"DynamicList[task=CompSize,resource=Speed,cores...",True,104.409735,0.022293,0.015228,4870.725405,111558.897850,893.462183,...,0.852713,0.00000,0.361629,0.000000,16,0.412833,0.000000,0.631771,0.000000,NaN
3,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-1250-10,"DynamicList[task=CompSize,resource=Speed,cores...",True,97.092569,0.018212,0.012316,4870.725405,111558.897850,89.257028,...,0.852713,0.00000,0.388882,0.000000,16,0.443945,0.000000,0.632432,0.000000,NaN
4,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-1250-100,"DynamicList[task=CompSize,resource=Speed,cores...",True,97.092929,0.020169,0.012595,4870.725405,111558.897850,89.346218,...,0.852713,0.00000,0.388881,0.000000,16,0.443943,0.000000,0.632430,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527467,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5612.012352,0.002131,0.000258,37701.250710,72918.129089,147.190401,...,1.000000,0.02257,0.746440,0.016962,9,0.746440,0.016962,0.837501,0.019326,NaN
527468,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-1250-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5612.019552,0.002046,0.000228,37701.250710,72918.129089,147.512601,...,1.000000,0.02257,0.746439,0.016962,9,0.746439,0.016962,0.837501,0.019326,NaN
527469,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-12500-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5606.004155,0.003421,0.001544,37701.250710,74002.393485,11.938948,...,1.000000,0.02257,0.747240,0.016980,9,0.747240,0.016980,0.838989,0.019362,NaN
527470,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-12500-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5606.004965,0.001334,0.000142,37701.250710,74002.393485,11.974498,...,1.000000,0.02257,0.747239,0.016980,9,0.747239,0.016980,0.838989,0.019362,NaN


# Finding DAG file paths

In [8]:
import os
import argparse

def find_file(target_filename, search_path="."):
    """
    Recursively search for files with the name target_filename starting from search_path.
    
    Parameters:
      search_path (str): The directory to start the search (absolute or relative).
      target_filename (str): The exact name of the file to search for.
      
    Returns:
      Full file path that match the target_filename, or False if
      none was found.
    """

    for root, dirs, files in os.walk(search_path):
        for file in files:
            if file == target_filename:
                return os.path.join(root, file)
    
    print("Error: File not found {}".format(target_filename))
    sys.exit(1)

In [9]:
# find_file('1000genome-chameleon-10ch-100k-001.json', os.getcwd() + "/../dags")

In [10]:
df["dag_path"] = df["dag"].apply(
    lambda file_name:
    find_file(file_name + ".json",
              search_path=os.getcwd() + "/../dags"))

In [11]:
# Make sure all dags were found
df[df["dag_path"] == False]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.max_memory_utilization,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path


In [12]:
df[df["dag_path"] == ""]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.max_memory_utilization,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path


# Now finding system file paths

In [13]:
df["system_path"] = df["system"].apply(
    lambda file_name:
    find_file(file_name + ".yaml",
              search_path=os.getcwd() + "/../systems"))

In [14]:
df[df["system_path"] == False]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path


In [15]:
df[df["system_path"] == ""]

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization,run_stats.memory_utilization,run_stats.used_resource_count,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path


# Measuring the DAGs properties
#### Properties: number of nodes and edges; graph density.


$$
DEN_{Undirected|Acyclic}=\frac{2|E|}{|V|*(|V|-1)}
$$

In [16]:
def calculate_dag_properties(dag):
    """
    Calculate the number of nodes, edges and density of a DAG.
        
    The DAG is assumed to have its tasks stored under:
        dag["workflow"]["tasks"]
        
    Each task is considered a node. The 'parents' field in each task lists its incoming edges.
    """
    # Retrieve tasks (nodes) from the DAG
    tasks = dag["workflow"]["tasks"]
    if not tasks:
        return 0, 0, 0  # If no tasks, then nodes, edges and density are all zero.

    num_nodes = len(tasks)

    # For graphs with fewer than 2 nodes, density is not defined (we return 0)
    if num_nodes < 2:
        return num_nodes, 0, 0.0
    
    # Sum up the edges from the length of the "parents" list for each task.
    num_edges = sum(len(task["parents"]) for task in tasks)
    
    # Compute density using the DAG density equation
    # (Actual number of edges vs maximum number of possible edges)
    density = (2 * num_edges) / (num_nodes * (num_nodes - 1))

    return num_nodes, num_edges, density


def calculate_dag_properties_helper(dag_path):
    with open(dag_path, "r") as f:
        dag = json.load(f)
        
    return calculate_dag_properties(dag)
    

In [17]:
unique_dags = df["dag_path"].unique()

In [18]:
dag_properties = []
for dag_path in unique_dags:
    properties = calculate_dag_properties_helper(dag_path)
    dag_properties.append([
        dag_path, properties[0], properties[1], properties[2]]) 

In [19]:
dag_properties_df = pd.DataFrame(dag_properties, columns=[
    "dag_path", "dag_nodes", "dag_edges", "dag_density"])

In [20]:
merged_df = pd.merge(df, dag_properties_df,
         left_on="dag_path", right_on="dag_path",
        how="left"
        )

In [21]:
merged_df

,dag,system,scheduler,completed,makespan,exec_time,run_stats.scheduling_time,run_stats.total_task_time,run_stats.total_network_traffic,run_stats.total_network_time,...,run_stats.cpu_utilization_used,run_stats.memory_utilization_used,run_stats.cpu_utilization_active,run_stats.memory_utilization_active,run_stats.expected_makespan,dag_path,system_path,dag_nodes,dag_edges,dag_density
0,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-125-10,"DynamicList[task=CompSize,resource=Speed,cores...",True,104.405775,0.018969,0.011636,4870.725405,111558.897850,892.481093,...,0.412848,0.000000,0.631785,0.000000,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,260,380,0.011286
1,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-125-100,"DynamicList[task=CompSize,resource=Speed,cores...",True,104.406135,0.018360,0.012827,4870.725405,111558.897850,892.570283,...,0.412847,0.000000,0.631783,0.000000,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,260,380,0.011286
2,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-125-1000,"DynamicList[task=CompSize,resource=Speed,cores...",True,104.409735,0.022293,0.015228,4870.725405,111558.897850,893.462183,...,0.412833,0.000000,0.631771,0.000000,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,260,380,0.011286
3,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-1250-10,"DynamicList[task=CompSize,resource=Speed,cores...",True,97.092569,0.018212,0.012316,4870.725405,111558.897850,89.257028,...,0.443945,0.000000,0.632432,0.000000,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,260,380,0.011286
4,1000genome-chameleon-10ch-100k-001,cluster-het-16-128-ConstantBandwidth-1250-100,"DynamicList[task=CompSize,resource=Speed,cores...",True,97.092929,0.020169,0.012595,4870.725405,111558.897850,89.346218,...,0.443943,0.000000,0.632430,0.000000,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,260,380,0.011286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527467,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-1250-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5612.012352,0.002131,0.000258,37701.250710,72918.129089,147.190401,...,0.746440,0.016962,0.837501,0.019326,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,104,152,0.028379
527468,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-1250-1000,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5612.019552,0.002046,0.000228,37701.250710,72918.129089,147.512601,...,0.746439,0.016962,0.837501,0.019326,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,104,152,0.028379
527469,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-12500-10,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5606.004155,0.003421,0.001544,37701.250710,74002.393485,11.938948,...,0.747240,0.016980,0.838989,0.019362,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,104,152,0.028379
527470,srasearch-chameleon-50a-005,cluster-hom-8-8-SharedBandwidth-12500-100,"DYTAS[navigation=Front,sorting=DFS,multicore=S...",True,5606.004965,0.001334,0.000142,37701.250710,74002.393485,11.974498,...,0.747239,0.016980,0.838989,0.019362,NaN,/home/hgsm/dytas-comparisons/analysis/../dags/...,/home/hgsm/dytas-comparisons/analysis/../syste...,104,152,0.028379


In [22]:
base_filename = "pre-processed-results-dataframe"

In [23]:
merged_df.to_parquet(
    "./data/{}.parquet".format(base_filename),
    engine="pyarrow",
    compression="gzip"
)

In [24]:
merged_df.to_pickle("./data/{}.pkl".format(base_filename))

In [25]:
merged_df.to_csv("./data/{}.csv".format(base_filename), index=False)